In [4]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from logger import Logger

In [5]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# MNIST dataset 
dataset = torchvision.datasets.MNIST(root='../../data', 
                                     train=True, 
                                     transform=transforms.ToTensor(),  
                                     download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset, 
                                          batch_size=100, 
                                          shuffle=True)

In [6]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size=784, hidden_size=500, num_classes=10):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet().to(device)

In [14]:
print(model)
dummy_input = torch.rand(784) #假设输入13张1*28*28的图片
model(dummy_input)
from tensorboardX import SummaryWriter

NeuralNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [15]:
with SummaryWriter(comment='NNet') as w:
    w.add_graph(model, (dummy_input, ))

In [8]:
logger = Logger('./logs')

# Loss and optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)  

data_iter = iter(data_loader)
# print(data_iter)
iter_per_epoch = len(data_loader)
# print(iter_per_epoch)#600
total_step = 10000

In [59]:
# Start training
for step in range(total_step):
    
    # Reset the data_iter
    if (step+1) % iter_per_epoch == 0:
        data_iter = iter(data_loader)
        #print(data_iter)
    
#     print(step)
    # Fetch images and labels
    images, labels = next(data_iter)
    images, labels = images.view(images.size(0), -1).to(device), labels.to(device)
#     print(labels.size())#torch.Size([100])
    
    # Forward pass
#     print(images.size())#torch.Size([100, 784])
    outputs = model(images)
    loss = criterion(outputs, labels)#计算损失
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Compute accuracy
    _, argmax = torch.max(outputs, 1)
    accuracy = (labels == argmax.squeeze()).float().mean()

    if (step+1) % 100 == 0:
        print ('Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(step+1, total_step, loss.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

        # 1. Log scalar values (scalar summary)
        info = { 'loss': loss.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            logger.scalar_summary(tag, value, step+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            logger.histo_summary(tag, value.data.cpu().numpy(), step+1)
            logger.histo_summary(tag+'/grad', value.grad.data.cpu().numpy(), step+1)

        # 3. Log training images (image summary)
        info = { 'images': images.view(-1, 28, 28)[:10].cpu().numpy() }

        for tag, images in info.items():
            logger.image_summary(tag, images, step+1)

Step [100/10000], Loss: 2.2068, Acc: 0.44
Step [200/10000], Loss: 2.1133, Acc: 0.62
Step [300/10000], Loss: 1.9831, Acc: 0.67
Step [400/10000], Loss: 1.8432, Acc: 0.75
Step [500/10000], Loss: 1.6672, Acc: 0.84
Step [600/10000], Loss: 1.6284, Acc: 0.76
Step [700/10000], Loss: 1.4899, Acc: 0.88
Step [800/10000], Loss: 1.3699, Acc: 0.81
Step [900/10000], Loss: 1.2305, Acc: 0.81
Step [1000/10000], Loss: 1.2140, Acc: 0.79
Step [1100/10000], Loss: 1.0486, Acc: 0.87
Step [1200/10000], Loss: 1.0591, Acc: 0.79
Step [1300/10000], Loss: 0.9131, Acc: 0.83
Step [1400/10000], Loss: 0.9403, Acc: 0.83
Step [1500/10000], Loss: 0.8940, Acc: 0.82
Step [1600/10000], Loss: 0.8860, Acc: 0.80
Step [1700/10000], Loss: 0.7919, Acc: 0.84
Step [1800/10000], Loss: 0.7651, Acc: 0.86
Step [1900/10000], Loss: 0.7044, Acc: 0.86
Step [2000/10000], Loss: 0.6755, Acc: 0.88
Step [2100/10000], Loss: 0.6063, Acc: 0.87
Step [2200/10000], Loss: 0.6927, Acc: 0.83
Step [2300/10000], Loss: 0.5720, Acc: 0.90
Step [2400/10000], L